# Segmentation

This notebook is for segmenting timelapse microscopy data, with associated sinhgle-cell labels and tracks, showing the infection of human macrophages with Mycobacterium Tuberculosis (Mtb), acquired on an Opera Phenix confocal microscope. 

In [3]:
import napari
import cellpose
from octopuslite import utils, tile

### Load experiment of choice

The Opera Phenix is a high-throughput confocal microscope that acquires very large 5-dimensional (TCZXY) images over several fields of view in any one experiment. Therefore, a lazy-loading approach is chosen to mosaic, view and annotate these images. This approach depends upon Dask and DaskFusion. The first step is to load the main metadata file (typically called `Index.idx.xml` and located in the main `Images` directory) that contains the image filenames and associated TCXZY information used to organise the images.

In [4]:
image_dir = '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Images/'
metadata_fn = '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Index.idx.xml'
metadata = utils.read_harmony_metadata(metadata_fn)

Reading metadata XML file...


Extracting HarmonyV5 metadata:   0%|          | 0/113400 [00:00<?, ?it/s]

Extracting metadata complete!


### View assay layout and mask information (optional)

The Opera Phenix acquires many time lapse series from a range of positions. The first step is to inspect the image metadata, presented in the form of an `Assaylayout/experiment_ID.xml` file, to show which positions correspond to which experimental assays.

In [5]:
metadata_path = '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Assaylayout/20210602_Live_cell_IPSDMGFP_ATB.xml'
utils.read_harmony_metadata(metadata_path, assay_layout=True)

Reading metadata XML file...
Extracting metadata complete!


Strain Compound Concentration ConcentrationEC
3 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA            60            EC50
  7     WT      RIF           0.1            EC50
  8     WT      INH          0.04            EC50
  9     WT      BDQ          0.02            EC50
4 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA            60            EC50
  7     WT      RIF           0.1            EC50
  8     WT      INH          0.04            EC50
  9     WT      BDQ          0.02            EC50
5 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA           400            EC99
  7     WT      RIF             2            EC99
  8     WT      INH             2            EC99
  9     WT      BDQ           2.5            EC99
6 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA           400            EC99
  7     WT      RIF             2            EC99
  8     WT      INH             2            EC99
  9     WT      BDQ           2.5            EC99

### Define row and column of choice

In [6]:
row = '3'
column = '7'
col = column

### Now to lazily mosaic the images using Dask prior to viewing them.

1x (75,2,3) [TCZ] image stack takes approximately 1 minute to stitch together, so only load the one field of view I want.

In [5]:
images = tile.compile_mosaic(image_dir, metadata, row, column)
images

Timepoint progress:   0%|          | 0/75 [00:00<?, ?it/s]

Channel progress:   0%|          | 0/2 [00:00<?, ?it/s]

Z-slice progress:   0%|          | 0/3 [00:00<?, ?it/s]

Z-slice progress:   0%|          | 0/3 [00:00<?, ?it/s]

Channel progress:   0%|          | 0/2 [00:00<?, ?it/s]

...

Z-slice progress:   0%|          | 0/3 [00:00<?, ?it/s]

Channel progress:   0%|          | 0/2 [00:00<?, ?it/s]

Z-slice progress:   0%|          | 0/3 [00:00<?, ?it/s]

Z-slice progress:   0%|          | 0/3 [00:00<?, ?it/s]

,Array,Chunk
Bytes,30.66 GiB,7.75 MiB
Shape,"(75, 2, 3, 6048, 6048)","(1, 1, 1, 2016, 2016)"
Count,20250 Tasks,4050 Chunks
Type,uint16,numpy.ndarray


# Only compile subset of whole image

In [2]:
import os
import pandas as pd
import numpy as np
from typing import Tuple, List, Dict, Union, Optional, Callable
ArrayLike = Union[
    np.ndarray, "dask.array.Array"
]
import dask
import dask.array as da
from functools import partial
from pathlib import Path
FilePath = Union[Path, str]
from tqdm.auto import tqdm

In [45]:
def compile_mosaic(
    image_directory: os.PathLike,
    metadata: pd.DataFrame,
    row: int,
    col: int,
    input_transforms: List[Callable[[ArrayLike], ArrayLike]] = None,
    set_plane = None,
    set_channel = None,
    set_time = None
    )->dask.array:

    """
    Uses the stitch function to compile a mosaic set of images that have been
    exported and fragmented from the Harmony software and returns a dask array
    that can be lazily loaded and stitched together on the fly.

    Parameters
    ----------
    image_directory : os.PathLike
        Location of fragmented images, typically located in a folder named
        "/Experiment_ID/Images" that was exported form the Harmony software.
    metadata : pd.DataFrame
        pd.DataFrame representation of the experiment metadata file, typically
        located in a file called "/Experiment_ID/Index.idx.xml". This metadata
        can be extracted and formatted using the `read_harmony_metadata`
        function in `utils.py`.
    row : str
        Each experiment will be conducted over a multiwell plate, so the row and
        column of the desired well needs to be defined as a string input. This
        input defines the row of choice.
    col : str
        Corresponding column of choice.
    input_transforms : List[Callable[[ArrayLike], ArrayLike]]
        Optional pre-processing transformations that can be applied to each
        image, such as a crop, a transpose or a background removal.
    plane : int
        Optional input to define a single plane to compile. If left blank then
        mosaic will be compiled over all planes available.
    channel : int
        Optional input to define a single channel to compile. If left blank then
        mosaic will be compiled over all channels available.
    time : int
        Optional input to define a single frame to compile. If left blank then
        mosaic will be compiled over all frames available.
    """
    ### extract some necessary information from the metadata before tiling
    channel_IDs = metadata['ChannelID'].unique() if channel == None else set_channel
    plane_IDs = metadata['PlaneID'].unique() if plane == None else set_plane
    timepoint_IDs = metadata['TimepointID'].unique() if time == None else set_time
    ### set a few parameters for the tiling approach
    chunk_fraction = 9
    load_transform_image = partial(load_image, transforms=input_transforms)
    ### clear empty arrays for organsing into dask arrays
    t_stack = []
    for time in tqdm(timepoint_IDs, leave = False, desc = 'Timepoint progress'):
        c_stack = []
        for channel in tqdm(channel_IDs, leave = False, desc = 'Channel progress'):
            z_stack = []
            for plane in tqdm(plane_IDs, leave = False, desc = 'Z-slice progress'):
                frame, chunk_info = tile.stitch(load_transform_image,
                                    metadata,
                                    image_directory,
                                    time,
                                    plane,
                                    channel,
                                    str(row),
                                    str(col),
                                    chunk_fraction,
                                    mask = False)
                ### collect stitched frames together into time stack
                z_stack.append(frame)
            ### stack channel series together (images)
            c_stack.append(z_stack)
        ### stack together timewise
        t_stack.append(c_stack)
    ### stack stitched dask arrays together into multidim image volumes
    images = da.stack([da.stack(c_stack, axis = 0) for c_stack in t_stack])

    return images

def load_image(
    file: FilePath, transforms: List[Callable[[ArrayLike], ArrayLike]] = None
) -> np.ndarray:
    img = imread(file)
    # if img.ndim == 2:
    #    img = np.expand_dims(img, axis=0)
    if transforms is not None:
        for t in transforms:
            img = t(img)
    return img

In [11]:
def load_image(
    file: FilePath, transforms: List[Callable[[ArrayLike], ArrayLike]] = None
) -> np.ndarray:
    img = imread(file)
    # if img.ndim == 2:
    #    img = np.expand_dims(img, axis=0)
    if transforms is not None:
        for t in transforms:
            img = t(img)
    return img

In [7]:
input_transforms = None

In [8]:
image_directory = image_dir

In [9]:
set_time = 0
set_channel = 1
set_plane = 1

In [47]:
for i in set_channel:
    print(i)

TypeError: 'int' object is not iterable

In [12]:
channel_IDs = metadata['ChannelID'].unique() if set_channel == None else range(set_channel)
plane_IDs = metadata['PlaneID'].unique() if set_plane == None else range(set_plane)
timepoint_IDs = metadata['TimepointID'].unique() if set_time == None else range(set_time)
### set a few parameters for the tiling approach
chunk_fraction = 9
load_transform_image = partial(load_image, transforms=input_transforms)
### clear empty arrays for organsing into dask arrays
t_stack = []
for time in tqdm(timepoint_IDs, leave = False, desc = 'Timepoint progress'):
    c_stack = []
    for channel in tqdm(channel_IDs, leave = False, desc = 'Channel progress'):
        z_stack = []
        for plane in tqdm(plane_IDs, leave = False, desc = 'Z-slice progress'):
            frame, chunk_info = tile.stitch(load_transform_image,
                                metadata,
                                image_directory,
                                time,
                                plane,
                                channel,
                                str(row),
                                str(col),
                                chunk_fraction,
                                mask = False)
            ### collect stitched frames together into time stack
            z_stack.append(frame)
        ### stack channel series together (images)
        c_stack.append(z_stack)
    ### stack together timewise
    t_stack.append(c_stack)
### stack stitched dask arrays together into multidim image volumes
images = da.stack([da.stack(c_stack, axis = 0) for c_stack in t_stack])

Timepoint progress: 0it [00:00, ?it/s]

ValueError: Need array(s) to stack

trying to both make it so that only a single slice can be tiled and also to do in a dask delayed manner

In [65]:
channel_IDs = metadata['ChannelID'].unique() if set_channel == None else set_channel
plane_IDs = metadata['PlaneID'].unique() if set_plane == None else set_plane
timepoint_IDs = metadata['TimepointID'].unique() if set_time == None else set_time
### set a few parameters for the tiling approach
chunk_fraction = 9
load_transform_image = partial(load_image, transforms=input_transforms)
### clear empty arrays for organsing into dask arrays
z_stack = dict()
for time in tqdm([timepoint_IDs], leave = False, desc = 'Timepoint progress'):
    z_stack[time] = dict()
    print(z_stack)
    for channel in tqdm([channel_IDs], leave = False, desc = 'Channel progress'):
        
        z_stack[time][channel] = [
        tile.stitch(load_transform_image,
                                metadata,
                                image_directory,
                                time,
                                plane,
                                channel,
                                str(row),
                                str(col),
                                chunk_fraction,
                                mask = False)[0]
        for plane in [plane_IDs]]
        
#         ### stack channel series together (images)
#         c_stack.append(z_stack)
#     ### stack together timewise
#     t_stack.append(c_stack)
# ### stack stitched dask arrays together into multidim image volumes
# # images = da.stack([da.stack(c_stack, axis = 0) for c_stack in t_stack])

Timepoint progress:   0%|          | 0/1 [00:00<?, ?it/s]

{0: {}}


Channel progress:   0%|          | 0/1 [00:00<?, ?it/s]

In [66]:
da.stack(z_stack[time]

{0: {1: [dask.array<fuse_func, shape=(6048, 6048), dtype=uint16, chunksize=(2016, 2016), chunktype=numpy.ndarray>]}}

In [67]:
channel_IDs

1

In [77]:
z_stack= dict()
for channel in tqdm([channel_IDs], leave = False, desc = 'Channel progress'):
        ### this currently is a z_stack but only a list
        z_stack[channel] = [
        tile.stitch(load_transform_image,
                                metadata,
                                image_directory,
                                time,
                                plane,
                                channel,
                                str(row),
                                str(col),
                                chunk_fraction,
                                mask = False)[0]
        for plane in [plane_IDs]]
        z_stack[channel] = da.stack(z_stack[channel], axis = 0)

Channel progress:   0%|          | 0/1 [00:00<?, ?it/s]

In [78]:
z_stack[1]

dask.array<stack, shape=(1, 6048, 6048), dtype=uint16, chunksize=(1, 2016, 2016), chunktype=numpy.ndarray>

In [85]:
for time in tqdm([timepoint_IDs], leave = False, desc = 'Timepoint progress'):
    z_stack[time] = dict()
    for channel in tqdm([channel_IDs], leave = False, desc = 'Channel progress'):
            ### this currently is a z_stack but only a list
            z_stack[time][channel] = [[
            tile.stitch(load_transform_image,
                                    metadata,
                                    image_directory,
                                    time,
                                    plane,
                                    channel,
                                    str(row),
                                    str(col),
                                    chunk_fraction,
                                    mask = False)[0]
            for plane in [plane_IDs]]
            for time in [timepoint_IDs]
            z_stack[time][channel] = da.stack(z_stack[time][channel], axis = 0)


Timepoint progress:   0%|          | 0/1 [00:00<?, ?it/s]

Channel progress:   0%|          | 0/1 [00:00<?, ?it/s]

In [87]:
z_stack

{1: dask.array<stack, shape=(1, 6048, 6048), dtype=uint16, chunksize=(1, 2016, 2016), chunktype=numpy.ndarray>,
 0: {1: dask.array<stack, shape=(1, 6048, 6048), dtype=uint16, chunksize=(1, 2016, 2016), chunktype=numpy.ndarray>}}

In [151]:
channel_IDs = metadata['ChannelID'].unique() if set_channel == None else set_channel
plane_IDs = metadata['PlaneID'].unique() if set_plane == None else set_plane
timepoint_IDs = metadata['TimepointID'].unique() if set_time == None else set_time

In [156]:
plane_IDs, channel_IDs, timepoint_IDs

(1, 1, 0)

In [143]:
timepoint_IDs = [0,1]

In [146]:
for time in [timepoint_IDs]:
    print(time)

[0, 1]


In [158]:
time, plane, channel

NameError: name 'plane' is not defined

In [162]:
plane_IDs

1

In [161]:
z_stack = [
#           [
#           [
            tile.stitch(load_transform_image,
                                    metadata,
                                    image_directory,
                                    time,
                                    plane,
                                    channel,
                                    str(row),
                                    str(col),
                                    chunk_fraction,
                                    mask = False)[0]
            for plane in [plane_IDs]]

#             for channel in [channel_IDs]]
#             for time in [timepoint_IDs]]
#     )
    # z_stack = da.stack(z_stack)

IndexError: list index out of range

In [160]:
z_stack = da.stack([
#           [
#           [
            tile.stitch(load_transform_image,
                                    metadata,
                                    image_directory,
                                    time,
                                    plane,
                                    channel,
                                    str(row),
                                    str(col),
                                    chunk_fraction,
                                    mask = False)[0]
            for plane in [plane_IDs]]

#             for channel in [channel_IDs]]
#             for time in [timepoint_IDs]]
    )
    # z_stack = da.stack(z_stack)

SyntaxError: invalid syntax (2427222640.py, line 14)

In [138]:
z_stack

dask.array<stack, shape=(1, 1, 6048, 6048), dtype=uint16, chunksize=(1, 1, 2016, 2016), chunktype=numpy.ndarray>

In [94]:
z_stack[0][1]

dask.array<stack, shape=(1, 1, 6048, 6048), dtype=uint16, chunksize=(1, 1, 2016, 2016), chunktype=numpy.ndarray>

In [110]:
z_stack[0][0][0]

dask.array<fuse_func, shape=(6048, 6048), dtype=uint16, chunksize=(2016, 2016), chunktype=numpy.ndarray>

In [114]:
type(z_stack)

list

In [115]:
len(z_stack)

1

In [117]:
z_stack.shape

AttributeError: 'list' object has no attribute 'shape'

In [57]:
z_stack = [
tile.stitch(load_transform_image,
                        metadata,
                        image_directory,
                        time,
                        plane,
                        channel,
                        str(row),
                        str(col),
                        chunk_fraction,
                        mask = False)[0]
for plane in [plane_IDs]]

In [58]:
z_stack

[dask.array<fuse_func, shape=(6048, 6048), dtype=uint16, chunksize=(2016, 2016), chunktype=numpy.ndarray>]

In [59]:
channel_IDs

1

In [62]:
z_stack

{1: [dask.array<fuse_func, shape=(6048, 6048), dtype=uint16, chunksize=(2016, 2016), chunktype=numpy.ndarray>]}

In [54]:
list(channel)

TypeError: 'int' object is not iterable

In [10]:
if plane channel, time == None

SyntaxError: invalid syntax (4283651572.py, line 1)

# Segment 
Let us start simple, only segmenting the lowest Z plane where the largest regions of cells are and only ch1 (GFP) where the GFP signal is.

In [6]:
!nvcc --version
!nvidia-smi

from cellpose import core, utils, io, models, metrics

use_GPU = core.use_gpu()
yn = ['NO', 'YES']
print(f'>>> GPU activated? {yn[use_GPU]}')

model = models.Cellpose(gpu=True, model_type='cyto')

def segment(img):
    masks, flows, styles, diams = model.eval(img, diameter=200, channels=[0,0],
                                             flow_threshold=None, cellprob_threshold=0)
    return masks

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
Fri Jan  6 13:50:14 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.86.01    Driver Version: 515.86.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    On   | 00000000:65:00.0  On |                  Off |
| 60%   76C    P2   171W / 300W |  37872MiB / 49140MiB |      3%      Default |
|                               |                      |                  N/A |
+

In [28]:
import dask.array as da
from tqdm.auto import tqdm

In [ ]:
mask_stack = []
for n, timepoint in tqdm(enumerate(images), total = len(images)):
    ### extract GFP channel and lowest Z plane from single time point
    gfp_z0_frame = timepoint[0,0,...]
    masks = segment(frame)
    mask_stack.append(masks)
mask_images = da.stack(mask_stack, axis = 0) 

In [31]:
mask_images = da.stack(mask_stack, axis = 0) 

# Testing different segmentation parameters 

In [36]:
### average cell diameter
diameters = [200, 250, 300]
### flow threshold, larger value means more ROIs (maybe ill fitting), lower means fewer ROIs 
flow_thresholds = [0.0, 0.4, 0.6, 0.8]
### cellprob_threshold, larger is is fewer ROIs, lower means more...? 
# cellprobs_thresholds = [-0.2, 0.0, 0.2]

In [34]:
import itertools

In [ ]:
mask_dict = dict()
params = list(itertools.product(diameters, flow_thresholds))
for diameter, flow_threshold in tqdm(params, total = len(params)):
    mask_stack = []
    for timepoint in tqdm(images, total = len(images), leave = False):
        ### extract GFP channel and lowest Z plane from single time point
        gfp_z0_frame = timepoint[0,0,...]
        masks, flows, styles, diams = model.eval(gfp_z0_frame, diameter=diameter, channels=[0,0],
                                             flow_threshold=flow_threshold, cellprob_threshold=0)        
        mask_stack.append(masks)
    mask_images = da.stack(mask_stack, axis = 0) 
    mask_dict[(diameter, flow_threshold)] = mask_images

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]

...

  0%|          | 0/75 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]

In [57]:
viewer = napari.Viewer()

viewer.add_image(images, 
                 channel_axis=1,
                 name=["macrophage", "mtb"],
                 colormap=["green", "magenta"],
                 contrast_limits=[[100, 2000], [100, 500]]
                 )
viewer.add_labels(mask_images, 
                 )

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


<Labels layer 'mask_images' at 0x7f995d5e5df0>